<h1>Capstone Project - The Battle of the Neighborhoods (Week 2)</h1>

<h3>Applied Data Science Capstone by IBM/Coursera</h3>

<h2>Introduction: Business Problem</h2>

In this project, we will try to identify the best possible location for a Japanese Restaurant in Clementi, Singapore. this report will be targeted to stakeholders who are interested to open a Japanese restaurant in the area of Clementi, Singapore.

Since there are a lot of Japanese restaurants in Singapore, we have to identify the best possible solution where there is a lack of Japanese restaurants in Singapore with lots of people residing in that area to maximise the number of customers to dine at that particular place.

We will use the data science power to generate the promising areas in the Clementi region based on the criteria.

<h2>Data</h2>

Based on defining our problem, factors that may influence our decisions are
<ul>
    <li>Number of exisiting Japanese restaurants in Clementi</li>
    <li>Number and distance between each Japanese restaurants in the neighbourhood</li>
    <li>Distance of neighbourhood to Clementi central, Clementi MRT station</li>
</ul>

From there, we would require the geometrics and foursquared API to perform this particular task

Lets define the libraries that we need to perform the task, first, we will gather the number of Japanese restaurants in the area to determine where there is a lack of Japanese restaurants in Clementi

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

CLIENT_ID = 'BZZ0EIXUGU1KROZDVCLUFJRLCR5NCHMPLQJYSFSIMDVFACRS' # your Foursquare ID
CLIENT_SECRET = '0APCH0S2PGFLVDLGDWD34AXG4WXG2DBMV4SFPO0XBIML5ZG4' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

We will find all Japanese restaurants in the area using the Foursquared API

In [3]:
address = 'Clementi, Singapore'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

1.3151003 103.7652311


In [33]:
search_query = 'Japanese'
radius = 1000
print(search_query + ' .... OK!')

Japanese .... OK!


In [34]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BZZ0EIXUGU1KROZDVCLUFJRLCR5NCHMPLQJYSFSIMDVFACRS&client_secret=0APCH0S2PGFLVDLGDWD34AXG4WXG2DBMV4SFPO0XBIML5ZG4&ll=1.3151003,103.7652311&v=20180604&query=Japanese&radius=1000&limit=30'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e09a285a2e538001b42621c'},
 'response': {'venues': [{'id': '4db3c7fc0437fa536a1d9dc1',
    'name': 'Ootoya Japanese Restaurant 大户屋',
    'location': {'address': '#03-53 The Clementi Mall',
     'crossStreet': '3155 Commonwealth avenue west',
     'lat': 1.3152849739411399,
     'lng': 103.76481023865026,
     'labeledLatLngs': [{'label': 'display',
       'lat': 1.3152849739411399,
       'lng': 103.76481023865026}],
     'distance': 51,
     'postalCode': '129558',
     'cc': 'SG',
     'city': 'Singapore',
     'country': 'Singapore',
     'formattedAddress': ['#03-53 The Clementi Mall (3155 Commonwealth avenue west)',
      '129558',
      'Singapore']},
    'categories': [{'id': '4bf58dd8d48988d111941735',
      'name': 'Japanese Restaurant',
      'pluralName': 'Japanese Restaurants',
      'shortName': 'Japanese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/japanese_',
       'suffix': '.png'},
      'primary': True}

In [36]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,name,referralId
0,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,4db3c7fc0437fa536a1d9dc1,#03-53 The Clementi Mall,SG,Singapore,Singapore,3155 Commonwealth avenue west,51,[#03-53 The Clementi Mall (3155 Commonwealth a...,"[{'label': 'display', 'lat': 1.315284973941139...",1.315285,103.764810,NaN,129558,Ootoya Japanese Restaurant 大户屋,v-1577689800
1,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,4d6e2caf792bb60c308e5ebe,@Clementi Mall,SG,Singapore,Singapore,NaN,144,"[@Clementi Mall, 129904, Singapore]","[{'label': 'display', 'lat': 1.316348868935473...",1.316349,103.764883,NaN,129904,Bachmann Japanese Restaurant,v-1577689800
2,"[{'id': '55a59bace4b013909087cb30', 'name': 'J...",False,5d594dae2605c800081089e6,NTUC Foodfare (@ The Clementi Mall),SG,Singapore,Singapore,3155 Commonwealth Avenue West,133,[NTUC Foodfare (@ The Clementi Mall) (3155 Com...,"[{'label': 'display', 'lat': 1.314808, 'lng': ...",1.314808,103.764069,NaN,122441,Japanese Food Goen,v-1577689800
3,"[{'id': '4bf58dd8d48988d1d1941735', 'name': 'N...",False,58142d8938fab8bf7817155a,Weng Kwang Coffee House,SG,Singapore,Singapore,"710 Clementi West Street 2, #01-247.",1111,[Weng Kwang Coffee House (710 Clementi West St...,"[{'label': 'display', 'lat': 1.305974, 'lng': ...",1.305974,103.761179,Clementi,120710,Ah Li Japanese Fusion Mee Pok,v-1577689800
4,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",False,4d3955eabf6d54810047c2e1,West Coast Highway,SG,Singapore,Singapore,NaN,1174,"[West Coast Highway, Singapore]","[{'label': 'display', 'lat': 1.311954014926437...",1.311954,103.755154,NaN,NaN,Bus Stop 20281 (Opp The Japanese Sec Sch),v-1577689800


In [37]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,id
0,Ootoya Japanese Restaurant 大户屋,Japanese Restaurant,#03-53 The Clementi Mall,SG,Singapore,Singapore,3155 Commonwealth avenue west,51,[#03-53 The Clementi Mall (3155 Commonwealth a...,"[{'label': 'display', 'lat': 1.315284973941139...",1.315285,103.764810,NaN,129558,4db3c7fc0437fa536a1d9dc1
1,Bachmann Japanese Restaurant,Japanese Restaurant,@Clementi Mall,SG,Singapore,Singapore,NaN,144,"[@Clementi Mall, 129904, Singapore]","[{'label': 'display', 'lat': 1.316348868935473...",1.316349,103.764883,NaN,129904,4d6e2caf792bb60c308e5ebe
2,Japanese Food Goen,Japanese Curry Restaurant,NTUC Foodfare (@ The Clementi Mall),SG,Singapore,Singapore,3155 Commonwealth Avenue West,133,[NTUC Foodfare (@ The Clementi Mall) (3155 Com...,"[{'label': 'display', 'lat': 1.314808, 'lng': ...",1.314808,103.764069,NaN,122441,5d594dae2605c800081089e6
3,Ah Li Japanese Fusion Mee Pok,Noodle House,Weng Kwang Coffee House,SG,Singapore,Singapore,"710 Clementi West Street 2, #01-247.",1111,[Weng Kwang Coffee House (710 Clementi West St...,"[{'label': 'display', 'lat': 1.305974, 'lng': ...",1.305974,103.761179,Clementi,120710,58142d8938fab8bf7817155a
4,Bus Stop 20281 (Opp The Japanese Sec Sch),Bus Stop,West Coast Highway,SG,Singapore,Singapore,NaN,1174,"[West Coast Highway, Singapore]","[{'label': 'display', 'lat': 1.311954014926437...",1.311954,103.755154,NaN,NaN,4d3955eabf6d54810047c2e1
5,Hakata Japanese Restaurant,Japanese Restaurant,NEWest,SG,Singapore,Singapore,1 West Coast Drive,838,"[NEWest (1 West Coast Drive), 128021, Singapore]","[{'label': 'display', 'lat': 1.316817, 'lng': ...",1.316817,103.757891,NaN,128021,58c2d2d437da1d5934edab35
6,The Japanese Sec Sch (20289),Bus Station,West Coast Highway,SG,Singapore,Singapore,NaN,1157,"[West Coast Highway, Singapore]","[{'label': 'display', 'lat': 1.313302155062489...",1.313302,103.754988,NaN,NaN,4f04546961afb3ab8a563bec
7,NTUC FoodFare,Food Court,"#04-20/22, The Clementi Mall",SG,Singapore,Singapore,3155 Commonwealth Ave West,135,"[#04-20/22, The Clementi Mall (3155 Commonweal...","[{'label': 'display', 'lat': 1.314875539367608...",1.314876,103.764037,NaN,128588,4d8e8bc15091a1cd1b0f9301
8,Shimizu Japanese Restaurant,Japanese Restaurant,23 Folkestone Road,SG,Singapore,Singapore,NaN,1060,"[23 Folkestone Road, 139599, Singapore]","[{'label': 'display', 'lat': 1.312512466282036...",1.312512,103.774406,NaN,139599,4b058813f964a5203fb022e3


In [38]:
dataframe_filtered.name

0               Ootoya Japanese Restaurant 大户屋
1                 Bachmann Japanese Restaurant
2                           Japanese Food Goen
3                Ah Li Japanese Fusion Mee Pok
4    Bus Stop 20281 (Opp The Japanese Sec Sch)
5                   Hakata Japanese Restaurant
6                 The Japanese Sec Sch (20289)
7                                NTUC FoodFare
8                  Shimizu Japanese Restaurant
Name: name, dtype: object

In [39]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Middle 

# add a red circle marker to represent the 
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Clementi',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Japanese restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

From the map above, we can determine that there is a lack of Japanese Restaurants in the North West area, we will confirm that there is no Japanese restaurants within 250 radius of that particular location

In [46]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
address = 'Sunset Way (Sunset Dr), Singapore'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

1.3232989 103.7691288


In [47]:
search_query = 'Japanese'
radius = 250
print(search_query + ' .... OK!')

Japanese .... OK!


In [48]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BZZ0EIXUGU1KROZDVCLUFJRLCR5NCHMPLQJYSFSIMDVFACRS&client_secret=0APCH0S2PGFLVDLGDWD34AXG4WXG2DBMV4SFPO0XBIML5ZG4&ll=1.3232989,103.7691288&v=20180604&query=Japanese&radius=250&limit=30'

In [49]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e09a49fb57e88001bc77fed'},
 'response': {'venues': []}}

In [72]:
if results['response']['venues'] == []:
    dataframe1 = []
else:
    venues = results['response']['venues']

    dataframe1 = json_normalize(venues)
    dataframe1

In [73]:
if dataframe1 != []:

    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')] + ['id']
    dataframe1_filtered = dataframe1.loc[:, filtered_columns]

    # function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']

        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

    # filter the category for each row
    dataframe1_filtered['categories'] = dataframe1_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe1_filtered.columns = [column.split('.')[-1] for column in dataframe1_filtered.columns]

    dataframe1_filtered

In [75]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Middle 

# add a red circle marker to represent the 
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Sunset Way',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

if dataframe1 != []:
# add the Japanese restaurants as blue circle markers
    for lat, lng, label in zip(dataframe1_filtered.lat, dataframe1_filtered.lng, dataframe1_filtered.categories):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            popup=label,
            fill = True,
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<h2>Results & Conclusion</h2>

From the data above, we can conclude that there is no Japanese restaurant in that particular area in the Sunset Way region, the Northwest area. Furthermore, there are lots of affluent people staying in that particular area. To add on, majority or all of them are able to afford to dine at a restaurant. We can finally conclude that having no Japanese restaurant in that particular area is an advantage of any stakeholders to start a Japanese Restaurant. Therefore, I strongly encourage any stakeholders to set up a Japanese restaurant in that particular area.